In [1]:
import requests
import lxml.html as lx
import time
import pandas as pd
import re
import concurrent.futures
import threading
import matplotlib.pyplot as plt
import numpy as np
from bs4 import BeautifulSoup
from collections import Counter

Getting all city state pair in United States from the numbeo website

In [2]:
prefix_url='https://www.numbeo.com/cost-of-living/country_result.jsp?country=United+States'
response = requests.get(prefix_url)
html_content = lx.fromstring(response.text)
links=html_content.xpath("//form//select[@name='city']/option/text()")



Getting Cost of living html for each city and state using the address

In [3]:
import requests

def get_html(address):
    address = address.strip()
    arr = address.split(',')
    if(len(arr)<2):
        return ''
    arr[0] = arr[0].replace(' ', '-')
    arr[0]=arr[0].strip()
    arr[1]=arr[1].strip()

    prefix_url = "https://www.numbeo.com/cost-of-living/in/"  # Replace this with your actual prefix URL

    urls_to_try = [
        f"{prefix_url}{arr[0]}-{arr[1]}-United-States",
        f"{prefix_url}{arr[0]}-{arr[1]}",
        f"{prefix_url}{arr[0]}"
    ]

    for url in urls_to_try:
        response = requests.get(url)
        # print(url)
        if response.status_code == 200:
            html_content = lx.fromstring(response.text)
            body=html_content.xpath("//div[@class='innerWidth']/h1/text()")[0]
            # print(body)
            if len(body)==0 or ('Cannot find city id for' not in body):
                print(url)
                return response.text
    return ''  # Or raise an exception if desired


In [4]:
html=get_html(links[1000])

https://www.numbeo.com/cost-of-living/in/Saint-Petersburg-FL


In [5]:
import lxml.html as lx

def get_field_from_html1(html_content, field):
    if(len(html_content)==0):
        return '0'
    meal = '0'
    # Parse HTML content using lxml
    tree = lx.fromstring(html_content)

    # Find all <tr> tags
    table_rows = tree.xpath('//tr')

    # Loop through each <tr> tag
    for row in table_rows:
        # Find the first <td> tag in the row
        first_td = row.find('td')
        if first_td is not None:
            # Check if the text of the first <td> tag is equal to the desired field
            if first_td.text_content().strip() == field:
                # Find the next <td> tag which contains the price
                price_td = first_td.getnext()
                if price_td is not None:
                    # Get the text inside the <span> tag
                    price_span = price_td.find('span[@class="first_currency"]')
                    if price_span is not None:
                        # Extract the dollar amount
                        dollar_amount = price_span.text_content().replace('&nbsp;', '').replace('$', '').strip()
                        if dollar_amount != '?':
                            meal = dollar_amount
                        break  # Exit loop since we found the desired value
    return meal


In [6]:
def get_field_from_html(html,field):
    soup = BeautifulSoup(html, 'html.parser')
    meal='0'
    # Find all <tr> tags
    table_rows = soup.find_all('tr')


    # Loop through each <tr> tag
    for row in table_rows:
        # Find the first <td> tag in the row
        first_td = row.find('td')
        if first_td:
            # Check if the text of the first <td> tag is "Meal, Inexpensive Restaurant"
            if first_td.get_text(strip=True) == field:
                # Find the next <td> tag which contains the price
                price_td = first_td.find_next('td')
                if price_td:
                    # Get the text inside the <span> tag
                    price_span = price_td.find('span', class_='first_currency')
                    if price_span:
                        # Extract the dollar amount
                        dollar_amount = price_span.get_text(strip=True).replace('&nbsp;', '')
                        dollar_amount=dollar_amount.replace('$','').strip()
                        if(dollar_amount!='?'):
                            meal=dollar_amount
                        break  # Exit loop since we found the desired value
    return meal



In [7]:
get_field_from_html('','Apartment (1 bedroom) in City Centre')

'0'

In [8]:
def get_data(html):
    data={}
    data['Milk']=get_field_from_html1(html,'Milk (regular), (1 gallon)')
    data['Rent']=get_field_from_html1(html,'Apartment (1 bedroom) in City Centre')
    data['Meal']=get_field_from_html1(html,'Meal, Inexpensive Restaurant')
    data['Water']=get_field_from_html1(html,'Water (12 oz small bottle)')
    return data


In [9]:
get_data(html)

{'Milk': '3.98', 'Rent': '1,900.00', 'Meal': '16.50', 'Water': '1.74'}

Calculate living expenses for each city,state pair in USA

In [ ]:
living_expense_data={}
import time
for address in links[1:]:
    time.sleep(3)
    html_content=get_html(address)
    data=get_data(html_content)
    living_expense_data[address]=data


https://www.numbeo.com/cost-of-living/in/Aberdeen-SD
https://www.numbeo.com/cost-of-living/in/Aberdeen-WA-United-States
https://www.numbeo.com/cost-of-living/in/Abilene
https://www.numbeo.com/cost-of-living/in/Abingdon-MD-United-States
https://www.numbeo.com/cost-of-living/in/Abingdon-VA-United-States
https://www.numbeo.com/cost-of-living/in/Acton-MA-United-States
https://www.numbeo.com/cost-of-living/in/Acworth-GA-United-States
https://www.numbeo.com/cost-of-living/in/Ada-OK-United-States
https://www.numbeo.com/cost-of-living/in/Adelanto-CA-United-States
https://www.numbeo.com/cost-of-living/in/Adrian-MI-United-States
https://www.numbeo.com/cost-of-living/in/Aiken-SC-United-States
https://www.numbeo.com/cost-of-living/in/Airway-Heights-WA-United-States
https://www.numbeo.com/cost-of-living/in/Akron
https://www.numbeo.com/cost-of-living/in/Alameda-CA-United-States
https://www.numbeo.com/cost-of-living/in/Alamogordo-NM-United-States
https://www.numbeo.com/cost-of-living/in/Albany-GA-Uni

In [ ]:
len(living_expense_data)

1283

Storing the data into .csv

In [10]:
import csv
with open('living_expenses.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=living_expense_data.keys())
    writer.writeheader()
    writer.writerow(living_expense_data)

In [ ]:
import json
import csv


# Convert dictionary to JSON
json_data = json.dumps(living_expense_data)

# Write JSON to CSV
with open('output_json_living_expense.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Key', 'Value'])  # Writing header row
    for key, value in json.loads(json_data).items():
        writer.writerow([key, value])


In [ ]:
import csv
import ast  # Import ast module to safely evaluate the string representation of the dictionary

# Initialize an empty dictionary
my_dict = {}

# Open the CSV file and load data into the dictionary
with open('output.csv', 'r', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        if len(row) == 2:  # Check if the row has exactly two values
            key, nested_dict_str = row
            # Safely evaluate the string representation of the nested dictionary
            nested_dict = ast.literal_eval(nested_dict_str)
            my_dict[key] = nested_dict
        else:
            print(f"Ignoring row {row}, it does not have exactly two values.")

# Print the dictionary
print(my_dict)


Ignoring row ['Aberdeen, SD', 'Aberdeen, WA', 'Abilene, TX', 'Abingdon, MD', 'Abingdon, VA', 'Acton, MA', 'Acworth, GA', 'Ada, OK', 'Addison IL', 'Adelanto, CA', 'Adrian, MI', 'Aiken, SC', 'Airway Heights, WA', 'Akron, OH', 'Alameda, CA', 'Alamogordo, NM', 'Albany, GA', 'Albany, NY', 'Albany, OR', 'Albert Lea, MN', 'Albrightsville, PA', 'Albuquerque, NM', 'Alexandria, LA', 'Alexandria, MN', 'Alexandria, VA', 'Alhambra, CA', 'Aliso Viejo, CA', 'Allegan, MI', 'Allen, TX', 'Allentown, PA', 'Alpena, MI', 'Alpharetta, GA', 'Alton, IL', 'Alvin, TX', 'Amarillo, TX', 'Ames, IA', 'Amherst, MA', 'Amherst, NY', 'Anaheim, CA', 'Anchorage, AK', 'Anderson, IN', 'Anderson, SC', 'Andover, MA', 'Angleton, TX', 'Ankeny, IA', 'Ann Arbor, MI', 'Anna, TX', 'Annapolis, MD', 'Antioch, CA', 'Apopka, FL', 'Apple Valley, CA', 'Apple Valley, MN', 'Appleton, WI', 'Archbold, OH', 'Arlington, KY', 'Arlington, TX', 'Arlington, VA', 'Arlington, WA', 'Artesia, NM', 'Ashburn, VA', 'Asheville, NC', 'Ashland, MA', 'Ashla

In [ ]:
living_expense_data

{'Aberdeen, SD': {'Milk': '0', 'Rent': '0', 'Meal': '0', 'Water': '0.79'},
 'Aberdeen, WA': {'Milk': '4.13',
  'Rent': '1,100.00',
  'Meal': '20.00',
  'Water': '2.47'},
 'Abilene, TX': {'Milk': '3.34',
  'Rent': '1,000.00',
  'Meal': '13.50',
  'Water': '1.70'},
 'Abingdon, MD': {'Milk': '2.97', 'Rent': '0', 'Meal': '25.00', 'Water': '0'},
 'Abingdon, VA': {'Milk': '0',
  'Rent': '550.00',
  'Meal': '10.00',
  'Water': '2.00'},
 'Acton, MA': {'Milk': '4.99', 'Rent': '0', 'Meal': '0', 'Water': '0'},
 'Acworth, GA': {'Milk': '3.00',
  'Rent': '800.00',
  'Meal': '27.50',
  'Water': '2.25'},
 'Ada, OK': {'Milk': '4.05', 'Rent': '800.00', 'Meal': '10.00', 'Water': '0'},
 'Addison IL': {'Milk': '0', 'Rent': '0', 'Meal': '0', 'Water': '0'},
 'Adelanto, CA': {'Milk': '4.54',
  'Rent': '0',
  'Meal': '20.00',
  'Water': '1.00'},
 'Adrian, MI': {'Milk': '0', 'Rent': '0', 'Meal': '10.00', 'Water': '1.25'},
 'Aiken, SC': {'Milk': '4.27', 'Rent': '0', 'Meal': '12.00', 'Water': '2.00'},
 'Airway H

In [ ]:
links

['--- Select city---',
 'Aberdeen, SD',
 'Aberdeen, WA',
 'Abilene, TX',
 'Abingdon, MD',
 'Abingdon, VA',
 'Acton, MA',
 'Acworth, GA',
 'Ada, OK',
 'Addison IL',
 'Adelanto, CA',
 'Adrian, MI',
 'Aiken, SC',
 'Airway Heights, WA',
 'Akron, OH',
 'Alameda, CA',
 'Alamogordo, NM',
 'Albany, GA',
 'Albany, NY',
 'Albany, OR',
 'Albert Lea, MN',
 'Albrightsville, PA',
 'Albuquerque, NM',
 'Alexandria, LA',
 'Alexandria, MN',
 'Alexandria, VA',
 'Alhambra, CA',
 'Aliso Viejo, CA',
 'Allegan, MI',
 'Allen, TX',
 'Allentown, PA',
 'Alpena, MI',
 'Alpharetta, GA',
 'Alton, IL',
 'Alvin, TX',
 'Amarillo, TX',
 'Ames, IA',
 'Amherst, MA',
 'Amherst, NY',
 'Anaheim, CA',
 'Anchorage, AK',
 'Anderson, IN',
 'Anderson, SC',
 'Andover, MA',
 'Angleton, TX',
 'Ankeny, IA',
 'Ann Arbor, MI',
 'Anna, TX',
 'Annapolis, MD',
 'Antioch, CA',
 'Apopka, FL',
 'Apple Valley, CA',
 'Apple Valley, MN',
 'Appleton, WI',
 'Archbold, OH',
 'Arlington, KY',
 'Arlington, TX',
 'Arlington, VA',
 'Arlington, WA',
 

In [ ]:
nested_dict=living_expense_data

Saving and retrieving the dictionary of living expenses

In [10]:
import csv

def flatten_dict(d):
    flat_list = []
    for key, inner_dict in d.items():
        flat_dict = {'Location': key}
        for k, v in inner_dict.items():
            flat_dict[k] = v
        flat_list.append(flat_dict)
    return flat_list

def unflatten_dict(data):
    nested_dict = {}
    for row in data:
        location = row.pop('Location')
        nested_dict[location] = row
    return nested_dict

# Flatten the nested dictionary
flat_data = flatten_dict(nested_dict)

# Write flattened data to CSV
with open('output_11.csv', 'w', newline='') as csvfile:
    fieldnames = ['Location', 'Milk', 'Rent', 'Meal', 'Water']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in flat_data:
        writer.writerow(row)




In [ ]:
def unflatten_dict(data):
    nested_dict = {}
    for row in data:
        location = row.pop('Location')
        nested_dict[location] = row
    return nested_dict

In [ ]:
# Read the CSV file and reconstruct the nested dictionary structure
nested_dict1 = unflatten_dict(csv.DictReader(open('output_11.csv')))

# Print the reconstructed nested dictionary
print(nested_dict1)

{'Aberdeen, SD': {'Milk': '0', 'Rent': '0', 'Meal': '0', 'Water': '0.79'}, 'Aberdeen, WA': {'Milk': '4.13', 'Rent': '1,100.00', 'Meal': '20.00', 'Water': '2.47'}, 'Abilene, TX': {'Milk': '3.34', 'Rent': '1,000.00', 'Meal': '13.50', 'Water': '1.70'}, 'Abingdon, MD': {'Milk': '2.97', 'Rent': '0', 'Meal': '25.00', 'Water': '0'}, 'Abingdon, VA': {'Milk': '0', 'Rent': '550.00', 'Meal': '10.00', 'Water': '2.00'}, 'Acton, MA': {'Milk': '4.99', 'Rent': '0', 'Meal': '0', 'Water': '0'}, 'Acworth, GA': {'Milk': '3.00', 'Rent': '800.00', 'Meal': '27.50', 'Water': '2.25'}, 'Ada, OK': {'Milk': '4.05', 'Rent': '800.00', 'Meal': '10.00', 'Water': '0'}, 'Addison IL': {'Milk': '0', 'Rent': '0', 'Meal': '0', 'Water': '0'}, 'Adelanto, CA': {'Milk': '4.54', 'Rent': '0', 'Meal': '20.00', 'Water': '1.00'}, 'Adrian, MI': {'Milk': '0', 'Rent': '0', 'Meal': '10.00', 'Water': '1.25'}, 'Aiken, SC': {'Milk': '4.27', 'Rent': '0', 'Meal': '12.00', 'Water': '2.00'}, 'Airway Heights, WA': {'Milk': '3.00', 'Rent': '0',

In [ ]:
nested_dict1